# Movie ratings - Bayesian vaerage

In [1]:
import random
import pandas as pd

In [2]:
df = pd.read_csv("./ml-25m/ratings.csv")
ndf = pd.read_csv("./ml-25m/movies.csv")

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


## Ranking movies on arithmetic mean (average)

In [4]:
all_aa = df.groupby('movieId')["rating"].mean().sort_values(ascending=False)

## Ranking movies on Bayesian average

In [5]:
avg_rating = df['rating'].mean()

In [6]:
avg_count_rating = df.groupby("movieId")['rating'].count().mean()

In [7]:
count_ratings = df.groupby("movieId").count()['rating']

In [8]:
# del count_ratings[175981]

In [9]:
adf = pd.DataFrame(count_ratings, columns=['rating'])

In [10]:
adf.rename(columns = {'rating':'ratings_count'}, inplace = True)

In [11]:
adf['avg_rating'] = df.groupby('movieId').mean()['rating']

In [12]:
adf['overall_avg_rating'] = avg_rating

In [13]:
adf['overall_avg_count_rating'] = avg_count_rating

In [14]:
adf['w'] = adf['ratings_count']/(adf['overall_avg_count_rating'] + adf['ratings_count'])

In [15]:
adf['aa'] = all_aa

In [16]:
adf['ba'] = adf['w'] * adf['avg_rating'] + (1-adf['w']) * adf['overall_avg_rating']

In [17]:
adf.head()

,ratings_count,avg_rating,overall_avg_rating,overall_avg_count_rating,w,aa,ba
movieId,,,,,,,
1,57309,3.893708,3.533854,423.393144,0.992666,3.893708,3.891069
2,24228,3.251527,3.533854,423.393144,0.982825,3.251527,3.256376
3,11804,3.142028,3.533854,423.393144,0.965373,3.142028,3.155596
4,2523,2.853547,3.533854,423.393144,0.856301,2.853547,2.951307
5,11714,3.058434,3.533854,423.393144,0.965117,3.058434,3.075019


In [18]:
adf.sort_values('aa', ascending=False).head()

,ratings_count,avg_rating,overall_avg_rating,overall_avg_count_rating,w,aa,ba
movieId,,,,,,,
136782,1,5.0,3.533854,423.393144,0.002356,5.0,3.537309
186119,1,5.0,3.533854,423.393144,0.002356,5.0,3.537309
137032,1,5.0,3.533854,423.393144,0.002356,5.0,3.537309
184643,1,5.0,3.533854,423.393144,0.002356,5.0,3.537309
137038,1,5.0,3.533854,423.393144,0.002356,5.0,3.537309


In [19]:
adf.sort_values('ba', ascending=False).head()

,ratings_count,avg_rating,overall_avg_rating,overall_avg_count_rating,w,aa,ba
movieId,,,,,,,
318,81482,4.413576,3.533854,423.393144,0.994831,4.413576,4.409028
858,52498,4.324336,3.533854,423.393144,0.992000,4.324336,4.318012
159817,1747,4.464797,3.533854,423.393144,0.804923,4.464797,4.283192
50,55366,4.284353,3.533854,423.393144,0.992411,4.284353,4.278658
1221,34188,4.261759,3.533854,423.393144,0.987767,4.261759,4.252854


In [20]:
adf['diff'] = adf['aa'] - adf['ba']

In [21]:
candidates = adf[(adf['ratings_count'] > 30)].sort_values(['diff'], ascending=False)[:50]

In [22]:
ndf[ndf.movieId.isin(list(candidates.index))][['movieId', 'title']]

,movieId,title
9204,27423,"O Auto da Compadecida (Dog's Will, A) (2000)"
9845,32906,"Ascent, The (Voskhozhdeniye) (1977)"
11620,53226,"Great War, The (Grande guerra, La) (1959)"
11985,56372,Tokyo Twilight (Tôkyô boshoku) (1957)
15513,81736,Hamlet (Gamlet) (1964)
17201,90341,Catching Hell (2011)
17261,90592,How to Die in Oregon (2011)
17534,91762,"Last Lions, The (2011)"
22677,115969,Generation War (2013)
23095,116975,Long Way Round (2004)


In [23]:
ndf[ndf.movieId == 119145].title.values[0]

'Kingsman: The Secret Service (2015)'

In [33]:
adf[adf.index == 119145]
adf.aa.rank(pct=True)[119145],adf.ba.rank(pct=True)[119145]

(0.8789608278151303, 0.9861127576337494)

In [38]:
def all_positions(movieId):
    df = pd.read_csv("./ml-latest-small/ratings.csv")
    df_ratings_for_movie = df[df.movieId == movieId].sort_values('timestamp', ascending=False)
    df = df.drop(df[df.movieId == movieId].index)
    
    pos = []
    
    count = 10
    for x in df_ratings_for_movie.itertuples():
        df = df.append([x], ignore_index=True)

        all_aa = df.groupby('movieId')["rating"].mean().sort_values(ascending=False)
        avg_rating = df['rating'].mean()
        avg_count_rating = df.groupby("movieId")['rating'].count().mean()
        count_ratings = df.groupby("movieId").count()['rating']
        adf = pd.DataFrame(count_ratings, columns=['rating'])
        adf.rename(columns = {'rating':'ratings_count'}, inplace = True)
        adf['avg_rating'] = df.groupby('movieId').mean()['rating']
        adf['overall_avg_rating'] = avg_rating
        adf['overall_avg_count_rating'] = avg_count_rating
        adf['w'] = adf['ratings_count']/(adf['overall_avg_count_rating'] + adf['ratings_count'])
        adf['aa'] = all_aa
        adf['ba'] = adf['w'] * adf['avg_rating'] + (1-adf['w']) * adf['overall_avg_rating']
        adf['ba_rank'] = adf.ba.rank(pct=True)
        adf['aa_rank'] = adf.aa.rank(pct=True)

        pos.append(adf[adf.index == movieId])
        
    return pos

poses = all_positions(119145)

In [39]:
len(poses)

36

In [40]:
pd.concat(poses)

,ratings_count,avg_rating,overall_avg_rating,overall_avg_count_rating,w,aa,ba,ba_rank,aa_rank
movieId,,,,,,,,,
119145,1,3.500000,3.501384,10.366207,0.087980,3.500000,3.501262,0.584070,0.565714
119145,2,4.250000,3.501399,10.366310,0.161730,4.250000,3.622470,0.828054,0.910222
119145,3,3.833333,3.501394,10.366413,0.224443,3.833333,3.575895,0.749537,0.725422
119145,4,3.875000,3.501399,10.366516,0.278425,3.875000,3.605419,0.810880,0.740693
119145,5,3.900000,3.501404,10.366619,0.325381,3.900000,3.631099,0.837567,0.747378
119145,6,3.750000,3.501399,10.366722,0.366598,3.750000,3.592535,0.802653,0.690868
119145,7,3.785714,3.501404,10.366824,0.403067,3.785714,3.616000,0.821524,0.708968
119145,8,3.812500,3.501409,10.366927,0.435566,3.812500,3.636909,0.870321,0.717400
119145,9,3.833333,3.501414,10.367030,0.464707,3.833333,3.655659,0.884718,0.725422
